# Centering a trajectory in the box
ref: [link](https://userguide.mdanalysis.org/stable/examples/transformations/center_protein_in_box.html), [link-on-the-fly](https://userguide.mdanalysis.org/stable/trajectories/transformations.html), [link-on-the-fly-blog](https://www.mdanalysis.org/2020/03/09/on-the-fly-transformations/)

Here we use MDAnalysis transformations to make a protein whole, center it in the box, and then wrap the water back into the box. We then look at how to do this on-the-fly.

In [1]:
import MDAnalysis as mda
import numpy as np
import nglview as nv

from MDAnalysis.tests.datafiles import TPR, XTC

## Loading files
The test files we will be working with here are trajectories of a adenylate kinase (AdK), a phosophotransferase enzyme. ([BDPW09])

For the step-by-step transformations, we need to load the trajectory into memory so that our changes to the coordinates persist. If your trajectory is too large for that, see the on-the-fly transformation section for how to do this out-of-memory.

In [2]:
u = mda.Universe(TPR, XTC, in_memory=True)

In [3]:
view = nv.show_mdanalysis(u)
view.add_representation('point', 'resname SOL')
view.center()
view

NGLWidget(max_frame=9)

In [4]:
from nglview.contrib.movie import MovieMaker
movie = MovieMaker(
    view,
    step=2,
    render_params={"factor": 2},  # average quality render
    output='original.gif',
)
movie.make()

IntProgress(value=0, description='Rendering ...', max=4)

## Unwrapping the protein
The first step is to “unwrap” the protein from the border of the box, to make the protein whole. MDAnalysis provides the `AtomGroup.unwrap` function to do this easily. Note that this function requires your universe to have bonds in it.

We loop over the trajectory to unwrap for each frame.

In [5]:
protein = u.select_atoms('protein')

for ts in u.trajectory:
    protein.unwrap(compound='fragments')

In [6]:
unwrapped = nv.show_mdanalysis(u)
unwrapped.add_representation('point', 'resname SOL')
unwrapped.center()
unwrapped

NGLWidget(max_frame=9)

## Centering in the box
The next step is to center the protein in the box. We calculate the center-of-mass of the protein and the center of the box for each timestep. We then move all the atoms so that the protein center-of-mass is in the center of the box.

If you don’t have masses in your trajectory, try using the `center_of_geometry`.

In [7]:
for ts in u.trajectory:
    protein_center = protein.center_of_mass(wrap=True)
    dim = ts.triclinic_dimensions
    box_center = np.sum(dim, axis=0) / 2
    u.atoms.translate(box_center - protein_center)

In [8]:
centered = nv.show_mdanalysis(u)
centered.add_representation('point', 'resname SOL')
centered.center()
centered

NGLWidget(max_frame=9)

## Wrapping the solvent back into the box
Luckily, MDAnalysis also has `AtomGroup.wrap` to wrap molecules back into the box. Our trajectory has dimensions defined, which the function will find automatically. If your trajectory does not, or you wish to use a differently sized box, you can pass in a `box` with dimensions in the form `[lx, ly, lz, alpha, beta, gamma]`.

In [9]:
not_protein = u.select_atoms('not protein')

for ts in u.trajectory:
    not_protein.wrap(compound='residues')

In [10]:
wrapped = nv.show_mdanalysis(u)
wrapped.add_representation('point', 'resname SOL')
wrapped.center()
wrapped

NGLWidget(max_frame=9)

# Doing all this on-the-fly
Running all the transformations above can be difficult if your trajectory is large, or your computational resources are limited. Use on-the-fly transformations to keep your data out-of-memory.

Some common transformations are defined in `MDAnalysis.transformations`.

In [11]:
import MDAnalysis.transformations as trans

In [12]:
u2 = mda.Universe(TPR, XTC)

protein2 = u2.select_atoms('protein')
not_protein2 = u2.select_atoms('not protein')

From version 1.0.0 onwards, the `MDAnalysis.transformations` module contains `wrap` and `unwrap` functions that correspond with the `AtomGroup` versions above. You can only use `add_transformations` once, so pass them all at the same time.

In [13]:
transforms = [trans.unwrap(protein2),
              trans.center_in_box(protein2, wrap=True),
              trans.wrap(not_protein2)]

u2.trajectory.add_transformations(*transforms)

In [14]:
otf = nv.show_mdanalysis(u2)
otf.add_representation('point', 'resname SOL')
otf.center()
otf

NGLWidget(max_frame=9)